In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
train=pd.read_csv('C:/Users/RAVINTHIRAN/Documents/Dept_stat_receipt_disposal_010112019.csv')
test=pd.read_csv('C:/Users/RAVINTHIRAN/Documents/datafile.csv')
train['Type']='Train'
test['Type']='Test'
fullData = pd.concat([train,test],axis=0) 

C:\Users\RAVINTHIRAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [5]:
fullData.columns 
fullData.head(10) 
fullData.describe()

,Disposals,Month,Pending Between 2 To 6 Months,Pending Between 6 To 12 Months,Pending Less Than 2 Months,Pending More Than 1 Year,Recetpts,Total Disposal (01.01.2016 to 01.11.2019),Total Pending as on 01.11.2019,Total Receipts (01.01.2016 to 01.11.2019),Year
count,5567.000000,5567.000000,124.000000,124.000000,124.000000,124.000000,5567.000000,124.000000,124.000000,124.000000,5567.000000
mean,688.464703,6.391054,953.524194,807.225806,1313.927419,4835.258065,837.197773,30635.120968,7909.935484,38545.056452,2017.489851
std,1591.735766,3.377380,2350.929854,2333.919157,3819.736530,14684.165297,1631.776291,63377.329252,21205.176508,72118.883037,1.102636
min,1.000000,1.000000,0.000000,0.000000,4.000000,0.000000,1.000000,19.000000,5.000000,77.000000,2016.000000
25%,48.000000,3.000000,8.000000,1.000000,68.000000,0.000000,82.000000,2399.500000,126.250000,3765.250000,2017.000000
50%,160.000000,6.000000,77.500000,18.500000,189.500000,12.000000,264.000000,8729.500000,507.000000,12712.500000,2017.000000
75%,606.500000,9.000000,350.500000,142.500000,1026.750000,353.500000,888.000000,30102.000000,2063.000000,41319.000000,2018.000000
max,25005.000000,12.000000,15758.000000,18278.000000,37806.000000,97928.000000,24977.000000,437024.000000,112301.000000,547042.000000,2019.000000


In [17]:
ID_col = ['REF_NO']
target_col = ["Disposals.Month"]
cat_cols = ['Disposals','Month','Pending Between 2 To 6 Months','Pending Between 6 To 12 Months','Pending Less Than 2 Months','Pending More Than 1 Year','Recetpts']
num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(ID_col)-set(target_col))
other_col=['Type'] 

In [9]:
fullData.isnull().any()

Disposals                                     True
Ministry/Department/State                     True
Month                                         True
Pending Between 2 To 6 Months                 True
Pending Between 6 To 12 Months                True
Pending Less Than 2 Months                    True
Pending More Than 1 Year                      True
Recetpts                                      True
Total Disposal (01.01.2016 to 01.11.2019)     True
Total Pending as on 01.11.2019                True
Total Receipts (01.01.2016 to 01.11.2019)     True
Type                                         False
Year                                          True
org_name                                      True
dtype: bool

In [10]:
num_cat_cols = num_cols+cat_cols

In [11]:
for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1 

In [12]:
fullData[num_cols] = fullData[num_cols].fillna(fullData[num_cols].mean(),inplace=True)

C:\Users\RAVINTHIRAN\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [19]:
for var in cat_cols:
 number = LabelEncoder()
 fullData[var] = number.fit_transform(fullData[var].astype('str'))


fullData["Disposals"] = number.fit_transform(fullData["Disposals"].astype('str'))

train=fullData[fullData['Type']=='Train']
test=fullData[fullData['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

In [20]:
features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(other_col))

In [23]:
x_train = Train[list(features)].values
y_train = Train["Month"].values
x_validate = Validate[list(features)].values
y_validate = Validate["Month"].values
x_test=test[list(features)].values

In [25]:
random.seed(100)
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train, y_train)

TypeError: fit() missing 1 required positional argument: 'y'

In [28]:
status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1])
data = auc(fpr, tpr)
print (data)

final_status = rf.predict_proba(x_test)
test["Account.Status"]=final_status[:,1]
test.to_csv('C:/Users/Analytics Vidhya/Desktop/model_output.csv',columns=['REF_NO','Account.Status'])

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.